In [1]:
!pip3 install mlxtend

In [5]:
import pandas as pd
import numpy as np

recipies = []
for r in range(100):
    ids = np.random.randint(1, 1001, 10)
    recipies.append({'doc': r, 'prod_ids': ids})

recipies_df = pd.DataFrame(recipies)
recipies_df.head(5)

,doc,prod_ids
0,0,"[407, 781, 95, 213, 938, 315, 80, 951, 898, 190]"
1,1,"[156, 784, 855, 503, 362, 759, 290, 852, 361, ..."
2,2,"[619, 417, 858, 895, 757, 562, 581, 125, 508, ..."
3,3,"[949, 802, 422, 831, 591, 535, 41, 31, 797, 903]"
4,4,"[247, 9, 997, 690, 93, 883, 737, 201, 684, 786]"


In [6]:
from mlxtend.preprocessing import TransactionEncoder

te=TransactionEncoder()
prods_1hot=te.fit(recipies_df['prod_ids']).transform(recipies_df['prod_ids'])
prods_1hot=pd.DataFrame(prods_1hot,columns=te.columns_)
prods_1hot


,3,9,10,11,12,13,15,17,18,20,...,989,991,992,993,994,996,997,998,999,1000
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
96,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
97,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
98,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
from mlxtend.frequent_patterns import fpgrowth
is_fp = fpgrowth(prods_1hot,min_support=0.000002,max_len=2,
                 use_colnames=True)

is_fp.head(5)

,support,itemsets
0,0.03,(898)
1,0.02,(190)
2,0.02,(315)
3,0.02,(407)
4,0.02,(95)


In [12]:
from mlxtend.frequent_patterns import association_rules
import pandas as pd

rules = association_rules(is_fp, metric="lift", min_threshold=10)
df1=rules.copy()

df2=df1['antecedents'].apply(iter).apply(next).reset_index()
df3=df1['consequents'].apply(iter).apply(next).reset_index()
df5=df1['lift'].reset_index()

df4=pd.merge(df2,df3)
df6=pd.merge(df4,df5)
reco_matrix=df6.groupby('antecedents')['consequents'].apply(list).reset_index()

reco_matrix.rename({'consequents': 'recommendations'}, axis=1)

print(df6)
df6.to_csv('./.data/reco_matrix.csv')


print("recommendation matrix size", reco_matrix.shape)
reco_matrix.head(5)


      index  antecedents  consequents       lift
0         0          898          908  11.111111
1         1          908          898  11.111111
2         2          898          190  16.666667
3         3          190          898  16.666667
4         4           42          190  16.666667
...     ...          ...          ...        ...
8529   8529          413          376  33.333333
8530   8530          413          759  33.333333
8531   8531          759          413  33.333333
8532   8532          736          413  16.666667
8533   8533          413          736  16.666667

[8534 rows x 4 columns]
recommendation matrix size (638, 2)


,antecedents,consequents
0,3,"[659, 537, 921, 838, 715, 301, 873, 927, 717]"
1,9,"[93, 883, 247, 684, 201, 737, 997, 690, 786]"
2,10,"[407, 450, 325, 742, 957, 66, 943, 477, 801, 7..."
3,11,"[646, 501, 802, 654, 210, 662, 458, 450, 83, 7..."
4,12,"[894, 627, 729, 419, 580, 411, 771, 321, 908, ..."


In [9]:
cart_items = [1, 6, 7]
reco_matrix[reco_matrix.antecedents.isin(cart_items)]

,antecedents,consequents
